In [12]:
import pandas as pd
from anthropic import Anthropic
import time
from tqdm.auto import tqdm
import signal
import sys
import os

class PromptProcessor:
    def __init__(self, df, api_key, output_path, model):
        self.df = df.copy()
        self.api_key = api_key
        self.output_path = output_path
        self.model = model
        self.client = Anthropic(api_key=api_key)
        self.is_interrupted = False
        
        # Initialize columns
        if 'ClaudeResponse' not in self.df.columns:
            self.df['ClaudeResponse'] = ''
            
        # Setup interrupt handler
        signal.signal(signal.SIGINT, self.handle_interrupt)
        signal.signal(signal.SIGTERM, self.handle_interrupt)

    def handle_interrupt(self, signum, frame):
        print("\n\nInterruption received. Shutting down gracefully...")
        self.is_interrupted = True

    def save_progress(self):
        self.df.to_csv(self.output_path, index=False)

    def process_prompts(self):
        pbar = tqdm(total=len(self.df), desc="Processing prompts")
        
        try:
            for idx in self.df.index:
                if self.is_interrupted:
                    break
                    
                # Skip if already processed
                if pd.notna(self.df.loc[idx, 'ClaudeResponse']):
                    pbar.update(1)
                    continue
                
                try:
                    message = self.client.messages.create(
                        model=self.model,
                        max_tokens=1024,
                        messages=[{
                            "role": "user",
                            "content": self.df.loc[idx, 'Prompt']
                        }]
                    )
                    
                    self.df.loc[idx, 'ClaudeResponse'] = message.content[0].text
                    
                except Exception as e:
                    print(f"\nError processing prompt {idx}: {str(e)}")
                    self.df.loc[idx, 'ClaudeResponse'] = f"Error: {str(e)}"
                
                # Save after every request
                self.save_progress()
                pbar.update(1)
                
        finally:
            pbar.close()
            self.save_progress()
            
        return self.df

def process_prompts_file(input_path, output_path, api_key, model="claude-3-sonnet-20240229"):
    """
    Process prompts from input CSV and save results.
    
    Args:
        input_path (str): Path to input CSV file containing prompts
        output_path (str): Path to save results
        api_key (str): Anthropic API key
        model (str): Claude model to use (default: claude-3-sonnet-20240229)
    """
    # Load existing progress or start new
    if os.path.exists(output_path):
        print(f"Resuming from {output_path}")
        df = pd.read_csv(output_path)
    else:
        print(f"Starting new processing from {input_path}")
        df = pd.read_csv(input_path)
    
    processor = PromptProcessor(df, api_key, output_path, model)
    return processor.process_prompts()


In [ ]:
df = pd.read_csv("../results/results_v1.csv")

# Your API key should be stored securely, not hardcodedß

# Process the prompts
results_df = process_prompts_file(input_path="../results/results_v1.csv", output_path='../results/results_claude.csv', api_key=api_key, model="claude-3-5-sonnet-20241022")

# Save results
results_df.to_csv('../results/results_claude.csv', index=False)

Resuming from existing progress in ../results/results_claude.csv


Processing prompts:   0%|          | 3/1681 [Successful: 0, Failed: 3, Retried: 0]


Progress saved to ../results/results_claude.csv
Backup saved to ../results/results_claude_20241122_114104.csv

Progress saved to ../results/results_claude.csv
Backup saved to ../results/results_claude_20241122_114131.csv


Interruption received. Saving progress and shutting down gracefully...

Progress saved to ../results/results_claude.csv
Backup saved to ../results/results_claude_20241122_114134.csv

Progress saved to ../results/results_claude.csv
Backup saved to ../results/results_claude_20241122_114134.csv

Processing Complete!
Total prompts processed: 24
Successful: 21
Failed: 3
Required retries: 0


SystemExit: 0

/home/hessel/miniconda3/envs/thesis/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
